In [21]:
#PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
import requests
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [22]:
# Define the API endpoint
api_url = "http://172.16.1.59:8080/v1/web/test12"

# Fetch data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a Python dictionary
    data = response.json()
    
    # Create a pandas DataFrame from the data
    # Assuming the API response is a list of dictionaries
    df = pd.DataFrame(data['data'])
    
else:
    print(f"Failed to fetch data: {response.status_code}")
    
display(df)
#pd.options.display.max_colwidth = 100

ConnectionError: HTTPConnectionPool(host='172.16.1.59', port=8080): Max retries exceeded with url: /v1/web/test12 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001FF0F224140>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [8]:
#PARAMETER
#WMA
weights = [0.1, 0.3, 0.6]

#EWMA
alpha_wma = 0.4

#SES & DES
alpha_ses = 0.65  # ubah nilai alpha (semakin besar semakin berat ke data terbaru)
beta_des = 0.45   # ubah nilai beta (semakin besar semakin cepat beradaptasi, kalo rendah bisa terjadi lag)

In [11]:
# get mean and standart deviation of first 12 data
df['mean_12'] = df['D'].apply(lambda x: np.mean(x[:12]))
df['std_12'] = df['D'].apply(lambda x: np.std(x[:12], ddof=1))

# get upper bound from mean and std
df['ub'] = df['mean_12'] + 1.5 * df['std_12']

# limit the original data to upper bound
df['clipped_d'] = df.apply(lambda row: np.clip(row['D'][:12], 0, row['ub']).tolist(), axis=1)
display(df)

,period,branch,agency,partno,D,mean_12,std_12,ub,clipped_d
0,2025-01-01,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",121.750000,14.448183,143.422275,"[100.0, 105.0, 110.0, 108.0, 115.0, 120.0, 125..."
1,2025-01-01,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",238.333333,23.094011,272.974349,"[200.0, 210.0, 220.0, 215.0, 230.0, 240.0, 245..."
2,2025-01-01,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",72.333333,14.729891,94.428170,"[50.0, 55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80...."
3,2025-01-01,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",340.000000,25.045413,377.568120,"[300.0, 310.0, 320.0, 315.0, 330.0, 340.0, 345..."
4,2025-01-01,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",527.500000,18.027756,554.541635,"[500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530..."
5,2025-01-01,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",173.583333,16.555462,198.416527,"[150.0, 155.0, 160.0, 158.0, 165.0, 170.0, 175..."
6,2025-01-01,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",423.583333,16.555462,448.416527,"[400.0, 405.0, 410.0, 408.0, 415.0, 420.0, 425..."
7,2025-01-01,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",103.583333,16.555462,128.416527,"[80.0, 85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 1..."
8,2025-01-01,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",290.000000,25.045413,327.568120,"[250.0, 260.0, 270.0, 265.0, 280.0, 290.0, 295..."
9,2025-01-01,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",640.000000,25.045413,677.568120,"[600.0, 610.0, 620.0, 615.0, 630.0, 640.0, 645..."


In [15]:
# Calculate Simple Moving Average
def calculate_ma(list):
    oldData = []
    newData = []
    for i in list:
        # store calculated data to old list
        oldData.append(i)
        newData.append(np.mean(oldData))
    return newData

df['ma'] = df['clipped_d'].apply(calculate_ma)
df['ma_result'] = df['ma'].apply(lambda x: x[-1:])
df['ma_result'] = df['clipped_d'].apply(lambda x: np.mean(x))
display(df)


,period,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result
0,2025-01-01,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",121.750000,14.448183,143.422275,"[100.0, 105.0, 110.0, 108.0, 115.0, 120.0, 125...","[100.0, 102.5, 105.0, 105.75, 107.6, 109.66666...",121.618523
1,2025-01-01,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",238.333333,23.094011,272.974349,"[200.0, 210.0, 220.0, 215.0, 230.0, 240.0, 245...","[200.0, 205.0, 210.0, 211.25, 215.0, 219.16666...",238.333333
2,2025-01-01,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",72.333333,14.729891,94.428170,"[50.0, 55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80....","[50.0, 52.5, 55.0, 55.75, 57.6, 59.66666666666...",72.333333
3,2025-01-01,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",340.000000,25.045413,377.568120,"[300.0, 310.0, 320.0, 315.0, 330.0, 340.0, 345...","[300.0, 305.0, 310.0, 311.25, 315.0, 319.16666...",340.000000
4,2025-01-01,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",527.500000,18.027756,554.541635,"[500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530...","[500.0, 502.5, 505.0, 507.5, 510.0, 512.5, 515...",527.461803
5,2025-01-01,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",173.583333,16.555462,198.416527,"[150.0, 155.0, 160.0, 158.0, 165.0, 170.0, 175...","[150.0, 152.5, 155.0, 155.75, 157.6, 159.66666...",173.451377
6,2025-01-01,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",423.583333,16.555462,448.416527,"[400.0, 405.0, 410.0, 408.0, 415.0, 420.0, 425...","[400.0, 402.5, 405.0, 405.75, 407.6, 409.66666...",423.451377
7,2025-01-01,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",103.583333,16.555462,128.416527,"[80.0, 85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 1...","[80.0, 82.5, 85.0, 85.75, 87.6, 89.66666666666...",103.451377
8,2025-01-01,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",290.000000,25.045413,327.568120,"[250.0, 260.0, 270.0, 265.0, 280.0, 290.0, 295...","[250.0, 255.0, 260.0, 261.25, 265.0, 269.16666...",290.000000
9,2025-01-01,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",640.000000,25.045413,677.568120,"[600.0, 610.0, 620.0, 615.0, 630.0, 640.0, 645...","[600.0, 605.0, 610.0, 611.25, 615.0, 619.16666...",640.000000
